## Basic Implementation of NN from scratch

In [43]:
import numpy as np
from numpy.typing import ArrayLike
from abc import ABC, abstractmethod
from typing import Callable, Tuple,Dict, List, Any

In [135]:
class VNNModule(ABC):
    """Abstract base class for neural network modules."""
    
    def __init__(self, name: str):
        super().__init__()
        self.module_name = name  # For debugging purposes
        self._output_size = np.NaN # Initialize output size as NaN

    @property
    @abstractmethod
    def output_size(self) -> Any:
        """Property to get the size of the output from this module."""
        return self._output_size
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}--{self.module_name}'
    
    def __call__(self, x: Any) -> Any:
        """Make the module callable."""
        return self.forward(x)
    
    @abstractmethod
    def summary(self) -> Dict[str, int]:
        """Summarize the parameters and structure of the module."""
        pass

    @abstractmethod
    def sanity_check(self, from_behind: Any) -> bool:
        """Checks whether input from the previous layer matches the expected shape."""
        pass

    @abstractmethod
    def forward(self, x: Any) -> Any:
        """Forward pass to process input x."""
        pass

    @abstractmethod
    def backward(self, from_front: Any) -> Any:
        """Backward pass for gradient calculation."""
        pass


### Layers

In [153]:
class VLinearLayer(VNNModule):
    def __init__(self, input_size:int, output_size: int, weight_initialize: np.ndarray = None, bias_initialize: np.ndarray = None):
        super().__init__(f'VLinearLayer mapping {input_size} -> {output_size}')
        self.input_size = input_size
        self._output_size = output_size
        #TODO: Replace random initialization with standard techniques
        self.WeightMatrix = np.random.rand(self.input_size, self.output_size) if weight_initialize is None else weight_initialize
        self.bias = np.random.rand(self.output_size) if bias_initialize is None else bias_initialize 
        #sanity check 
        if self.WeightMatrix.shape != (input_size, output_size):
            raise ValueError('Input and output sizes did not matched with the weight matrix size.')
        if self.bias.shape != (output_size,):
            raise ValueError('Output size did not matched with the bias vector size.')
        #for computation of gradients
        self.forward_acc = None
        self.backward_acc = None
        #for summary
        self.summary_dict =  {'input_count': self.input_size, 
                              'output_cout': self._output_size, 
                              'total_params': self.input_size*self._output_size+self.input_size}
    
    @property
    def output_size(self):
        return self._output_size
        
    def forward(self, x: np.ndarray) -> np.ndarray:
        #forward pass is simple to implement
        #Linear layer has equation == f(x) = W.x + b
        if x.shape[0] != self.input_size:
            raise ValueError(f"Input size {x.shape[0]} does not match expected input size {self.input_size}.")
        out = x @ self.WeightMatrix
        out += self.bias
        self.forward_acc  = out
        return out
    
    def summary(self) -> Dict[str, int]:
        return self.summary_dict
    
    def sanity_check(self, from_behind: VNNModule) -> bool:
        """Returns true if shapes are compatible"""
        print(f'From Behind: {from_behind} and Forward: {self.WeightMatrix.shape[0]}')
        return from_behind.output_size == self.WeightMatrix.shape[0]

    def clear_accumulations(self) -> None:
        self.forward_acc = None
        self.backward_acc = None

    def backward(self, from_front: np.ndarray) -> Tuple[np.ndarray]:
        #compute gradient of weights and biases and multiply them with from front
        pass

In [154]:
class VConvLayer(VNNModule):
    # Implement convolutional layer here
    pass

class VPoolingLayer(VNNModule):
    # Implement pooling layer here
    pass

class VDropOutLayer(VNNModule):
    # Implement dropout layer here
    pass

In [155]:
from typing import Any, Dict


class VSigmoidLayer(VNNModule):
    def __init__(self, output_size):
        super().__init__(f'VSigmoid Layer({output_size})')
        self.forward_acc = None
        self.backward_acc = None
        self._output_size = output_size

    @property
    def output_size(self):
          return self._output_size

    def forward(self, x: np.ndarray) -> np.ndarray:
            #forward pass 
            out = 1.0/(1.0+np.e**(-x))
            self.forward_acc = out
            return  out
    
    def clear_accumulations(self) -> None:
            self.forward_acc = None
            self.backward_acc = None
    
    def summary(self) -> Dict[str, int]:
          return {} #activation function

    def sanity_check(self, from_behind: VNNModule) -> bool:
          print(f'From Behind: {from_behind} and Forward: {self._output_size}')
          return from_behind.output_size == self._output_size #no need for checking in activation function
    
    def backward(self, from_front:np.ndarray) -> np.ndarray:
        sigmoid_grad = self.forward_acc * (1 - self.forward_acc)
        self.backward_acc = sigmoid_grad * from_front
        return self.backward_acc

In [156]:
from typing import Any, Dict

class VReLuLayer(VNNModule):
    def __init__(self, output_size):
        # Initialize the layer
        super().__init__(f'VReLU Layer({output_size})')
        self.err_term = 0.00004  # Values smaller than this are treated as 0
        self.forward_acc = None  # Stores forward pass result
        self.backward_acc = None  # Stores backward pass result
        self._output_size = output_size
        
    @property
    def output_size(self):
          return self._output_size
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Forward pass for ReLU."""
        out = np.maximum(0, x)  # NumPy's vectorized ReLU
        self.forward_acc = out  # Store for backward pass
        return out

    def clear_accumulations(self) -> None:
        """Clear stored forward and backward results."""
        self.forward_acc = None
        self.backward_acc = None

    def sanity_check(self, from_behind: VNNModule) -> bool:
        print(f'From Behind: {from_behind} and Forward: {self._output_size}')
        return from_behind.output_size == self._output_size #no need for checking in activation function
    
    def summary(self) -> Dict[str, int]:
        return {}
    
    def backward(self, from_front: np.ndarray) -> np.ndarray:
        """Backward pass for ReLU."""
        # Derivative of ReLU: 1 where forward_acc > err_term, else 0
        relu_grad = (self.forward_acc > self.err_term).astype(np.float32)
        
        # Element-wise multiplication with incoming gradient
        self.backward_acc = relu_grad * from_front
        return self.backward_acc

### Sequence wise execution

In [157]:
class VSequence(VNNModule):
    """
    VSequence orchestrates the execution of layers sequentially,
    handling both forward and backward passes.
    """

    def __init__(self, layers: List[VNNModule]):
        super().__init__(f'VSequence--{len(layers)}')
        self._output_size = layers[-1].output_size
        self.layers = layers
        print(self.layers)
        self.forward_acc = None  # Stores forward pass result
        self.backward_acc = None  # Stores backward pass result
        if self.sanity_check() == False:
            raise ValueError(f'Sanity check failed, due to incompatible layers: {self.layers}')

    @property
    def output_size(self):
          return self._output_size
    
    def sanity_check(self) -> bool:
        """Check if all layers are compatible in sequence."""
        for i in range(1, len(self.layers)):
            if not self.layers[i].sanity_check(self.layers[i - 1]):
                return False
        return True

    def summary(self) -> List[Dict[str, int]]:
        """Return a summary of each layer."""
        return [layer.summary() for layer in self.layers]

    def clear_accumulations(self) -> None:
        """Clear stored forward and backward results."""
        self.forward_acc = None
        self.backward_acc = None
        for layer in self.layers:
            layer.clear_accumulations()

    def forward(self, x: Any) -> Any:
        """Run the forward pass through all layers."""
        out = x
        for layer in self.layers:
            out = layer.forward(out)
        self.forward_acc = out
        return out

    def backward(self, from_front: Any) -> Any:
        """Run the backward pass through all layers in reverse order."""
        grad = from_front
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        self.backward_acc = grad
        return self.backward_acc

In [160]:
model = VSequence(layers=[
    VLinearLayer(5, 10),
    VSigmoidLayer(10),
    VLinearLayer(10,5),
    VReLuLayer(5),
    VLinearLayer(5, 1),
    VSigmoidLayer(1)
    ])

[VLinearLayer--VLinearLayer mapping 5 -> 10, VSigmoidLayer--VSigmoid Layer(10), VLinearLayer--VLinearLayer mapping 10 -> 5, VReLuLayer--VReLU Layer(5), VLinearLayer--VLinearLayer mapping 5 -> 1, VSigmoidLayer--VSigmoid Layer(1)]
From Behind: VLinearLayer--VLinearLayer mapping 5 -> 10 and Forward: 10
From Behind: VSigmoidLayer--VSigmoid Layer(10) and Forward: 10
From Behind: VLinearLayer--VLinearLayer mapping 10 -> 5 and Forward: 5
From Behind: VReLuLayer--VReLU Layer(5) and Forward: 5
From Behind: VLinearLayer--VLinearLayer mapping 5 -> 1 and Forward: 1


In [161]:
model.forward(np.random.randn(5))

array([0.99999738])